<a href="https://colab.research.google.com/github/kimhwijin/TensorflowWithKeras/blob/master/RNN/seq2seq_translate.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import nltk
import numpy as np
import re
import shutil
import tensorflow as tf
import os
import unicodedata
import zipfile
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction


In [3]:
def preprocessing_sentence(sent):
    sent = "".join([c for c in unicodedata.normalize("NFD", sent) if unicodedata.category(c) != "Mn"])
    #문자열 앞에 r 이붙으면 그대로 반환 r'abcd\n' = abcd\n
    sent = re.sub(r"([!.?])", r" \1", sent)
    #알파벳 또는 ! ? 제외하고 공백으로 치환
    sent = re.sub(r"[^a-zA-Z!.?]+", r" ", sent)
    #공백문자를 띄어쓰기 한칸으로 변경
    sent = re.sub(r"\s+", " ", sent)
    sent = sent.lower()
    return sent

def download_and_read(url, num_sent_pairs=30000):

    local_file = url.split('/')[-1]
    drive_path = "drive/MyDrive/Datasets/anki-fra-eng"
    data_path = os.join.path(drive_file, local_file)
    if not os.path.exists(data_path):
        os.system('-wget -O {:s} {:s}'.format(local_file, url))
        with zipfile.ZipFile(data_path, 'r') as zip_ref:
            zip_ref.extractall(data_path)
    file_path = os.join.path(data_path, 'fra_txt')
    en_sents, fr_sents, fr_sents_out = [], [], []

    with open(file_path, 'r') as fin:
        for i , line in enumerate(fin):
            en_sent, fr_sent = line.strip().split('\t')
            en_sent = [w for w in preprocessing_sentence(en_sent).split()]
            fr_sent = preprocessing_sentence(fr_sent)
            fr_sent_in = [w for w in ("BOS" + fr_sent).split()]
            fr_sent_out = [w for w in (fr_sent + "EOS").split()]
            en_sents.append(en_sent)
            fr_sents_in.append(fr_sent_in)
            fr_sents_out.append(fr_sent_out)
            if i >= num_sent_pairs - 1:
                break
    return en_sents, fr_sents_in, fr_sents_out
